# Sam Jeffery

# Date Started: 1/21/2025

# Healthcare Data Analysis and Visualization Pipeline



Objective

This dataset contains 1338 rows of insured data, where the Insurance charges are given against the following attributes of the insured: Age, Sex, BMI, Number of Children, Smoker and Region. The attributes are a mix of numeric and categorical variables. There are no missing or undefined values in the dataset. 

https://www.kaggle.com/datasets/teertha/ushealthinsurancedataset

My last project was only using Python. This time I will use:
Python to load data
MySQL to perform queries
then use Tableau for visualizations.


Deliverables:

    Python scripts for data cleaning and preprocessing.
    SQL database with healthcare data and optimized schema.
    SQL queries showcasing insightful analyses.
    Tableau dashboard visualizing healthcare data insights.


In [38]:
# Basic Imports. Now I need to find a dataset.

import numpy as np
import pandas as pd

df = pd.read_csv(r"insurance.csv")

df.shape

(1338, 7)

In [39]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [40]:
df.info()

df.notna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


age         1338
sex         1338
bmi         1338
children    1338
smoker      1338
region      1338
charges     1338
dtype: int64

Now, since I did a quick view of the data, and we know from the data dictionary that there are no null rows or columns, so nothing really needs to be cleaned.

I will still be using Python, but I will be connecting to a SQL Database and running all of my queries from VSCode to make it easier to look at!

In [41]:
import mysql.connector
# login files config kept seperately
import login_files as lf

mydb = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = lf.password
)

mycursor = mydb.cursor()

# db creation
#mycursor.execute("CREATE DATABASE claims")

# Setting DB Connection parameters, updating cursor
mydb.database = 'claims'
mycursor = mydb.cursor()



Now that the DB is created, we need to think about the Data Model.

We have a total of 7 attributes in the dataframe. I am going to split them up in to three tables. The first table will be the Patient Table.

Patient will contain information on a patient, such as:

patient_id -- the primary key for a patient
age -- int
sex -- varchar(10)
region_id -- int, FK to the region_id in regions

Then, we will store HealthData. This will be called HealthData

patient_id -- the foreign key for patient identification
bmi -- float
children -- int
smoker -- bool
charges -- float

Finally, we will have the region table. This table reduces data redundancy.

Region_id -- int, PK
region_name -- varchar(255)

In [42]:
mycursor.execute("DROP TABLE IF EXISTS healthdata")
mycursor.execute("DROP TABLE IF EXISTS patients")
mycursor.execute("DROP TABLE IF EXISTS regions")
mydb.commit()

In [43]:
# Creating the Regions table
mycursor.execute("CREATE TABLE regions(region_id INT AUTO_INCREMENT PRIMARY KEY, region_name VARCHAR(255))")

In [44]:
# Creating the Patients Table
mycursor.execute("CREATE TABLE patients (patient_id INT AUTO_INCREMENT PRIMARY KEY, age INT, sex VARCHAR(10), region_id INT, FOREIGN KEY (region_id) REFERENCES regions(region_id))")

In [45]:
# Creating the HealthDetails Table
mycursor.execute("CREATE TABLE healthdata (patient_id INT AUTO_INCREMENT, FOREIGN KEY (patient_id) REFERENCES patients(patient_id),bmi FLOAT, children INT, smoker BOOLEAN, charges INT)")

In [46]:
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

('healthdata',)
('patients',)
('regions',)


Now that we have verified our tables are created, it's time to load data.

There is some slight cleaning to do first, however. Most importantly, Smoker needs to be changed to be a proper boolean value.



In [47]:
# Replace yes and no with 1s and 0s
df['smoker'].replace(('yes', 'no'), (1, 0), inplace=True)

# We also changed the name of region to region_name, so let's accomodate for that
df = df.rename(columns={'region': 'region_name'})

regions_df = pd.DataFrame({
    'region_name': ['southwest', 'southeast', 'northwest', 'northeast'],
    'region_id': [1, 2, 3, 4]
})

region_mapping = regions_df.set_index('region_name')['region_id'].to_dict()
df['region_id'] = df['region_name'].map(region_mapping)


C:\Users\njhdt\AppData\Local\Temp\ipykernel_10076\1086562167.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['smoker'].replace(('yes', 'no'), (1, 0), inplace=True)
C:\Users\njhdt\AppData\Local\Temp\ipykernel_10076\1086562167.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['smoker'].rep

In [48]:
# Clean databases before loading (this wouldn't be used in a live env)
mycursor.execute("DELETE FROM patients")
mycursor.execute("DELETE FROM healthdata")
mycursor.execute("DELETE FROM regions")
mydb.commit()


In [49]:
def add_data(df):

    '''
    Splits the original data frame in to three based on which attributes they have, and adds them to the database.

    Args: pandas datframe
    
    '''
    
    # Split the single DF to three seperate DFs
    try:
        patients_df = df[['age', 'sex', 'region_id']]
        healthdata_df = df[['bmi', 'children', 'smoker',  'charges']]
        # Only concerned about unique regions
        regions_df = pd.DataFrame({'region_name': df['region_name'].unique()})
    
        # Dict to hold Dataframes
        df_dict = {"regions":regions_df, "patients":patients_df, "healthdata":healthdata_df}
        # Now we need to do a few things on each data frame.
        for table_name, df in df_dict.items():
            # First, to do this iteratively, we make placeholders for the cursor.
            placeholders = ', '.join(['%s'] * len(df.columns))
            # Then, convert the DF to a tuple so it can be handled by SQL
            tuples = [tuple(x) for x in df.to_numpy()]
            # After that, we get the column names
            columns = ', '.join(df.columns)
            query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
            mycursor.executemany(query, tuples)
            mydb.commit()

    except Exception as e:
        return(f"Error, ({e})")
    return "Data Successfully Added!"
add_data(df)

'Data Successfully Added!'

Now that our data is fully loaded, we can begin querying.